In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import os
import gzip
import csv

In [3]:
pip install matplotlib


In [4]:
from six.moves import urllib

In [5]:
URL_PATH='http://ai.stanford.edu/~btaskar/ocr/letter.data.gz'
DOWNLOADED_FILENAME='letter.data.gz'

In [6]:
def download_data():
    if not os.path.exists(DOWNLOADED_FILENAME):
        filename,_=urllib.request.urlretrieve(URL_PATH,DOWNLOADED_FILENAME)
    print(URL_PATH,DOWNLOADED_FILENAME)

In [7]:
download_data()

http://ai.stanford.edu/~btaskar/ocr/letter.data.gz letter.data.gz


In [8]:
def read_lines():
    with gzip.open(DOWNLOADED_FILENAME,'rt') as f:
        reader=csv.reader(f,delimiter='\t')
        lines=list(reader)
        return lines

In [9]:
lines=read_lines()
len(lines)

52152

In [10]:
lines[1][:15]

['2', 'm', '3', '1', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

In [11]:
def get_features_labels(lines):
    lines=sorted(lines,key=lambda x:int(x[0]))
    data,target=[],[]
    next_id=-1
    word=[]
    word_pixels=[]
    for line in lines:
        next_id=int(line[2])
        
        pixels=np.array([int(x) for x in line[6:134]])
        pixels=pixels.reshape((16,8))
        
        word_pixels.append(pixels)
        
        word.append(line[1])
        
        
        if next_id==-1:
            data.append(word_pixels)
            target.append(word)
        
            word=[]
            word_pixels=[]
        
    return data,target 

In [12]:
data , target=get_features_labels(lines)

In [13]:
len(data)  , len(target)

(6877, 6877)

In [14]:
def pad_features_labels(data,target):
    max_length=max(len(x) for x in target)
    padding=np.zeros((16,8))
    
    
    data=[x+ ([padding]*(max_length-len(x))) for x in data]
    
    target=[x+ ([' ']*(max_length-len(x))) for x in target]
    
    return np.array(data), np.array(target)

In [15]:
padded_data , padded_target =pad_features_labels(data,target)

In [16]:
len(padded_data) , len(padded_target)

(6877, 6877)

In [17]:
padded_target[:10]

array([['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' '],
       ['o', 'm', 'm', 'a', 'n', 'd', 'i', 'n', 'g', ' ', ' ', ' ', ' ',
        ' ']], dtype='<U1')

In [18]:
padded_target[200:210]

array([['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' '],
       ['m', 'b', 'r', 'a', 'c', 'e', 's', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ']], dtype='<U1')

In [19]:
word_length=len(padded_target[0])
word_length

14

In [20]:
padded_data.shape

(6877, 14, 16, 8)

In [21]:
padded_data.shape[:2]

(6877, 14)

In [22]:
padded_data.shape[:2]+(-1,)

(6877, 14, -1)

In [23]:
reshaped_data=padded_data.reshape(padded_data.shape[:2]+(-1,))
reshaped_data.shape

(6877, 14, 128)

In [24]:
padded_target.shape

(6877, 14)

In [25]:
padded_target.shape +(26,)

(6877, 14, 26)

In [26]:
one_hot_target=np.zeros(padded_target.shape + (26,))
one_hot_target.shape

(6877, 14, 26)

In [27]:
for index,letter in np.ndenumerate(padded_target):
    if letter:
        print(letter,index)
        one_hot_target[index][ch(letter) - ch('a')]  =  1

o (0, 0)


NameError: name 'ch' is not defined

In [28]:
one_hot_target[0][3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [29]:
one_hot_target[0][1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [30]:
shuffle_indices=np.random.permutation(len(reshaped_data))
shuffled_data=reshaped_data[shuffle_indices]
shuffled_target=one_hot_target[shuffle_indices]
                                      

In [31]:
split=int(0.66*len(shuffled_data))

In [32]:
train_data=shuffled_data[:split]
train_target=shuffled_target[:split]

test_data=shuffled_data[split:]
test_target=shuffled_target[split:]

In [33]:
train_data.shape

(4538, 14, 128)

In [34]:
test_data.shape

(2339, 14, 128)

In [35]:
train_target.shape

(4538, 14, 26)

In [36]:
_,num_steps,num_inputs=train_data.shape

In [37]:
num_classes=train_target.shape[2]

In [38]:
num_steps,num_inputs,num_classes

(14, 128, 26)

In [39]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [40]:
X=tf.placeholder(tf.float64, [None,num_steps,num_inputs])
y=tf.placeholder(tf.float64, [None,num_steps,num_classes])

In [41]:
used=tf.sign(tf.reduce_max(tf.abs(X),reduction_indices=2))
length=tf.reduce_sum(used,reduction_indices=1)
sequence_length=tf.cast(length,tf.int32)

In [42]:
sequence_length

<tf.Tensor 'Cast:0' shape=(?,) dtype=int32>

In [43]:
num_neurons=300

In [44]:
cell=tf.nn.rnn_cell.GRUCell(num_neurons)


Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.


In [45]:
output,_=tf.nn.dynamic_rnn(cell,X,dtype=tf.float64,sequence_length=sequence_length)
output.shape

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


TensorShape([Dimension(None), Dimension(14), Dimension(300)])

In [53]:
weight=tf.Variable(tf.truncated_normal([num_neurons,num_classes],stddev=0.01,dtype=tf.float64))
bias=tf.Variable(tf.constant(0.1,shape=[num_classes],dtype=tf.float64))

In [54]:
flattened_output=tf.reshape(output,[-1,num_neurons])
flattened_output

<tf.Tensor 'Reshape_2:0' shape=(?, 300) dtype=float64>

In [55]:
logits=tf.matmul(flattened_output,weight)+bias

In [56]:
logits_reshaped=tf.reshape(logits,[-1,num_steps,num_classes])

In [57]:
cross_entropy=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [58]:
loss=tf.reduce_mean(cross_entropy)

In [59]:
mistakes=tf.not_equal(tf.argmax(y,2),tf.argmax(logits_reshaped,2))
mistakes=tf.cast(mistakes,tf.float64)

In [60]:
mask=tf.sign(tf.reduce_max(tf.abs(y),reduction_indices=2))

In [61]:
mistakes*=mask

In [64]:
mistakes=tf.reduce_sum(mistakes,reduction_indices=1)
mistakes/=tf.cast(sequence_length,tf.float64)

ValueError: Invalid reduction dimension 1 for input with 1 dimensions. for 'Sum_3' (op: 'Sum') with input shapes: [?], [] and with computed input tensors: input[1] = <1>.

In [65]:
error=tf.reduce_mean(mistakes)

In [67]:
optimizer=tf.train.RMSPropOptimizer(0.002)
gradient=optimizer.compute_gradients(loss)

optimize=optimizer.apply_gradients(gradient)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [68]:
def batched(data,target,batch_size):
    epoch=0
    offset=0
    
    while True:
        old_offset=offset
        offset=(offset+batch_size)%(target.shape[0]-batch_size)
        
        if offset<old_offset:
            shuffled_indices=np.random.permutation(len(data))
            
            data=data[shuffled_indices]
            target=target[shuffled_indices]
            
            epoch+=1
            
            batch_data=data[offset:[offset+batch_size],:]
            batch_target=target[offset:[offset+batch_size],:]
            
            yield batch_data,batch_target,epoch

In [69]:
batch_size=20
batches=batched(train_data,train_target,batch_size)

In [70]:
batches

<generator object batched at 0x000001803E607548>

In [71]:
epochs=5

In [72]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for index,batch in enumerate(batches):
        batch_data=batch[0]
        batch_target=batch[1]
        
        epoch=batch[2]
        
        
        if epoch>epochs:
            break
        feed={X:batch_data,y:batch_target}
        train_error,_=sess.run((error,optimize),feed)
        
        print('{}:  {:3.6f}%'.format(index+1,100*train_error))
    
    test_feed={X:test_data,y:test_target}
    test_error,_=sess.run((error,optimize),test_feed)
    
    print('Test Error:  {:3.6f}%'.format(100*test_error))

TypeError: slice indices must be integers or None or have an __index__ method